In [9]:
import pickle
import pandas as pd
import numpy as np
import sys
sys.path.insert(0, '../..')
import os
import subprocess
from importlib import reload
import pickle

In [19]:
colnames=['article_id', 'title', 'abstract']
        
df_abst = pd.read_csv("/home/hilal/Documents/chem_relation_extractor/data/chemprot_test_gs/chemprot_test_abstracts_gs.tsv", sep="\t", names=colnames, header=None)
df_abst.head()

,article_id,title,abstract
0,10076535,Androgen antagonistic effect of estramustine p...,"Estramustine phosphate is used frequently, alo..."
1,10089512,Static Laue diffraction studies on acetylcholi...,Acetylcholinesterase (AChE) is one of nature's...
2,10201371,Synthesis of thrombin-inhibiting heparin mimet...,Unwanted side effects of pharmacologically act...
3,10220509,Selective inhibition of cyclooxygenase 2 spare...,BACKGROUND & AIMS: The critical role of cycloo...
4,10351938,Inhibition of cPLA2 translocation and leukotri...,We examined the effect of the highly lipophili...


In [27]:
with open('../../pickles/changed_id_article_frame.pickle', 'rb') as f:
    df_abst = pickle.load(f)

In [20]:
for i in df_abst.index:
    file = open( "input_test_gs/" + str(df_abst.loc[i,'article_id']) + ".txt", "w")
    article_id = df_abst.loc[i,'article_id']
    inp = list(df_abst[df_abst['article_id'] == article_id]['title'] + ' ' + df_abst[df_abst['article_id'] == article_id]['abstract'])[0]
    file.write(inp)


In [21]:
for i in df_abst.index:
    cmd = "./geniass "+ "input_test_gs/" + str(df_abst.loc[i,'article_id']) + ".txt " + "output_test_gs/" + str(df_abst.loc[i,'article_id']) + "_o.txt"
    output = subprocess.check_output(cmd, shell = True)
    lines = output.decode().split("\n")
    output = [i.split('\t')[0] for i in lines if i.split('\t')[0]]
    file.close()


In [22]:
df_sentence = df_abst[['article_id']].copy()
df_sentence['sent_id'] = 0
df_sentence['sentence'] = ''

for i in df_abst.index:
    file = open("output_test_gs/"+str(df_abst.loc[i,'article_id']) + "_o.txt", "r")
    sent_list = file.readlines()
    df_temp = pd.DataFrame({'article_id':df_abst.loc[i,'article_id'], 'sent_id':np.arange(len(sent_list)) })
    for j in range(len(sent_list)):
        df_temp.loc[df_temp['sent_id'] == j, 'sentence'] = sent_list[j].replace('\n', '')
    df_sentence = df_sentence.append([df_temp])
    
df_sentence.drop_duplicates(subset = ['article_id', 'sent_id'], keep = 'last', inplace = True)
   

In [23]:
with open('../../pickles/geniass_sentence_split_frame_test_gs.pickle', 'wb' ) as f:
    pickle.dump(df_sentence, f, protocol = pickle.HIGHEST_PROTOCOL)